In [1]:
import numpy as np
import pandas as pd

In [2]:
verbose = False

In [3]:
# read stats
stats = pd.read_excel('stats.xls').set_index('Name')

In [4]:
class Character:
    def __init__(self, data):
        self.name = data.name
        for k, v in data.items():
            setattr(self, k.lower(), v)
    
    def __str__(self):
        return self.name
    
    def __repr__(self):
        return self.name
    
    def roll_initiative(self):
        roll = np.random.randint(20) + 1 + (self.dexterity - 10) // 2
        self.initiative = roll
        return roll
    
    def attack(self, target):
        # roll against target armor class
        roll = np.random.randint(20) + 1 + self.melee_bonus
        if verbose:
            print('Roll d20+%d = %d' % (self.melee_bonus, roll))
        
        if roll < target.armor_class:
            if verbose:
                print('%s missed %s' % (self, target))
            return
        
        # roll damage
        n_dice, damage = self.melee_damage.split('d')
        n_dice = int(n_dice)
        if '+' in damage:
            damage_die, damage = [int(s) for s in damage.split('+')]
        elif '-' in damage:
            damage_die, damage = [int(s) for s in damage.split('-')]
            damage *= -1
        
        for n in range(n_dice):
            damage += np.random.randint(damage_die) + 1
        
        damage = max(0, damage)
        
        
        target.hit_points -= damage
        
        if verbose:
            print('%s hit %s and dealt %d damage' % (self, target, damage))

        return

In [5]:
goblin_wins = 0
player_wins = 0

for n in range(1000):
    player = Character(stats['Defaulticus'])
    goblin = Character(stats['Goblin'])
    
    # roll initiative
    player.roll_initiative()
    goblin.roll_initiative()
    
    # determine order
    lineup = np.array([player, goblin])
    order = np.argsort([x.initiative for x in lineup])[::-1]
    lineup = lineup[order]
    
    while (player.hit_points > 0) and (goblin.hit_points > 0):
        if goblin.hit_points > 0:
            player.attack(goblin)

        if goblin.hit_points > 0:
            goblin.attack(player)
    
    if player.hit_points > 0:
        player_wins += 1
    
    if goblin.hit_points > 0:
        goblin_wins += 1

print('Goblin: %d, Player: %d' % (goblin_wins, player_wins))

Goblin: 52, Player: 948
